## 1-8. 내가 직접 만들지 않고 가져다 쓰는 강아지 고양이 분류기 (1) 어려운 문제는 직접 풀지 말자, 똑똑한 누군가가 이미 풀어놨을 테니!

### 이미지 분류 대회인 ILSVRC와 데이터셋 ImageNet

지금까지 직접 분류 모델을 설계하고 구조를 짠 후, 학습시켜서 강아지와 고양이를 분류해 보았습니다.
하지만 사실 그것은 처음 접하는 과정이기 때문에 직접 실습을 해본 것일 뿐, 현실에서 이미지를 분류할 때는 절대 그렇게 모델을 직접 만들어서 사용하지 않습니다. (😧 네..? 그럼 우린 뭘 한 거죠?)

딥러닝 세계에서 이미지 분류는 굉장히 전통이 깊은 주제입니다. 이미 매우 많은 사람들이 시도해왔고, 정확도를 올리기 위해서도 굉장히 오랜 시간 동안 노력해왔죠.

심지어는 이미지 분류를 전문적으로 시도해 온 컴퓨터 비전 분야의 "올림픽"과 같은 대회도 있습니다.
바로 ILSVRC(ImageNet Large Scale Visual Recognition Challenge) 이죠. 다음 세 개의 글을 읽고 ILSVRC가 뭔지 알아봅시다.

ILSVRC 대회 (이미지넷 이미지 인식 대회) 역대 우승 알고리즘들
https://bskyvision.com/425

CNN-Imagenet에 쓰인 주요 모델
https://www.sallys.space/blog/2018/01/26/cnn-imagenet/

CNN 성능은 어떻게 평가하게 되나요?
https://89douner.tistory.com/59


Q. ILSVRC는 무엇의 약자인가요? 또, 어떤 것을 하는 대회인가요? 왜 지금은 종료되었나요?

ILSVRC는 ImageNet Large Scale Visual Recognition Challenge의 약자로, 이미지를 분류하는 대회이다.
2017년 이후 대회에서 활약하는 모델들의 성능이 사실상 사람을 뛰어 넘었다고 판단되었기 때문에 종료되었다.

Q. ILSVRC에서 분류해야 하는 데이터셋의 이름은 무엇인가요? 이 데이터셋은 어떤 데이터로 이루어져 있나요?

대회에서 사용되는 데이터셋은 ImageNet이라는 데이터셋으로, 총 1000개의 카테고리와 수백 만의 이미지 데이터로 이루어져 있다.

이미지 분류는 ILSVRC 대회를 통해 굉장히 빠르게 발전해왔습니다. 2017년 대회를 마지막으로 대회는 종료가 되었으며, 마지막 해의 우승 모델인 SeNet은 2.3%의 아주 낮은 오차율을 보였죠. 총 1,000개의 카테고리가 있는 수백만의 데이터를 분류하는 데 2.3%밖에 틀리지 않았다니, 정말 놀랍지 않나요?

그래서 우리는 이 대회에서 사용되었던 모델을 활용해 볼 것입니다.

하지만 잠깐, 의문점이 생기지 않나요?

대회에서 사용된 모델은 분명 1,000개의 카테고리를 맞추어야 하는 모델이었고, 학습한 데이터도 우리가 사용하는 데이터와 다른데, 어떻게 그 모델을 우리의 강아지 고양이 분류기에 활용할 수 있는 걸까요?

이러한 의문을 풀기 위해 한 가지 알고 가야 할 개념이 있습니다.
바로 거인의 어깨에 서서 문제를 푼다는, Transfer Learning(전이학습) 이죠.

### Transfer Learning

다음 글을 읽어보며 Transfer Learning의 개념을 파악하고, 다음 질문들에 답해봅시다.

Transfer Learning｜학습된 모델을 새로운 프로젝트에 적용하기

https://jeinalog.tistory.com/13

Q. 일반적인 CNN 모델은 두 파트로 나눠집니다. 두 파트는 각각 무엇인가요? 각 파트는 무슨 역할을 하나요?

합성곱층과 풀링층이 여러겹 쌓여있는 Convolutional base : 이미지로부터 특징을 효과적으로 추출하는 것 (feature extraction)
완전 연결 계층으로 이루어진 Classifier : 추출된 특징을 잘 학습해서 이미지를 알맞은 카테고리로 분류하는 것 (image classification)

Q. Convolutional base은 낮은 레벨의 계층과 높은 레벨의 계층으로 분리해볼 수 있습니다. 각 계층은 무엇을 의미하며, 이미지에서 어떤 특징을 추출하나요?

낮은 레벨의 계층은 input에 가까운 계층으로, 이미지에서 주로 일반적인(general) 특징을 추출한다.
높은 레벨의 계층은 output에 가까운 계층으로, 보다 구체적이고 특유한 특징을 추출한다.

Q. 전이학습은 사전에 학습된 모델을 내 프로젝트에 맞게 재정의하는 것입니다. 글에서는 세 가지 전략을 소개합니다. 각 전략은 무엇인가요? 그리고, 어떤 경우에 각 전략이 적합한가요?

전략 1은 전체 모델을 새로 학습시키는 것이다. 이 경우에는 사전학습 모델의 구조만 사용한다. 모델을 완전히 새로 학습시켜야하므로, 큰 사이즈의 데이터셋과 좋은 컴퓨팅 연산 능력이 있을 때 적합하다.

전략 2는 Convolutional base의 일부분은 고정시킨 상태로, 나머지 계층과 classifier만 새로 학습시키는 것이다. 데이터셋의 크기에 따라 얼마나 많은 계층을 새로 학습시킬지 달라지는데, 데이터의 양이 많을수록 더 많이 새로 학습시키고, 데이터의 양이 적을수록 학습시키는 부분을 적게 한다.

전략 3은 Convolutional base는 고정시키고, classifier만 새로 학습시키는 것이다. 이 경우는 convolutional base는 건들지 않고 그대로 두면서 특징 추출 메커니즘으로 활용하고, classifier만 재학습시키는 방법이다. 컴퓨팅 연산능력이 부족하거나 데이터셋이 작을 때 고려해볼 수 있다.

Q. 그렇다면 우리는 어떤 전략을 쓰는 것이 좋을까요? 데이터셋의 크기를 고려해서 생각해봅시다.

우리가 사용해 볼 모델은 ILSVRC에서 쓰인 모델로, 해당 대회는 수백 만 장의 데이터로 학습된 모델이다. 그에 비해 우리가 새로 학습시킬 데이터는 강아지와 고양이는 약 2만 장으로, 비교적 매우 작은 데이터셋이다. 따라서 모델을 새로 학습시키기보다는, 가져오는 모델은 이미지에서 특징을 추출하는 메커니즘으로 활용하고, classifier만 새로 학습시키는 전략 3이 적절하다.

Transfer Learning이 무엇인지 조금은 와닿으시나요?

간단히 말하자면, Transfer Learning은 누군가 이미 잘 만들어놓은 모델을 가져와서, 내 입맛에 맞게 일부분만 새로 학습시켜 사용하는 것입니다. Transfer Learning에는 여러 가지 전략이 있지만 우리는 그 중 "classifier" 부분만 새로 학습시키는, 즉 적은 부분만 새로 학습시키는 전략을 활용할 것입니다.

바로 다음 그림 중 세 번째 그림처럼 convolutional base는 모두 가만히 두고(freeze), 아래의 classifier 부분만 새로 학습시키는 거죠.

![](https://d3s0tskafalll9.cloudfront.net/media/images/e-4-transfer.max-800x600_oqjiLxu.png)

## 1-9. 내가 직접 만들지 않고 가져다 쓰는 강아지 고양이 분류기 (2) 아주 똑똑한 모델을 간단히 가져오는 방법

### 사전 학습 모델인 VGG16을 가져오고, 모델의 구조 확인하기

그렇다면 이제 설명은 그만, 바로 실행해 보죠!

우리는 2014년의 ILSVRC 물체 위치 찾기(localization) 태스크에서 우승한 VGG 모델을 사용해 보겠습니다.
2014년 이후 뛰어난 성능을 보이는 다양한 새로운 모델이 개발되었지만, VGG는 단순한 구조임에도 꽤나 괜찮은 성능을 보여서 아직까지 많은 이미지 처리 분야에서 사랑받는 모델 중 하나입니다.

다음의 간단한 코드로 VGG 모델을 base_model의 변수로 불러오겠습니다. 이전 스텝에 언급한 대로, VGG 모델을 가져올 때, output에 가까운 높은 레벨에 있는 3개의 FC 레이어는 제외하고 불러와야 하므로 include_top=False 옵션을 주었다는 것을 눈여겨 봐주세요.
텐서플로우 케라스에서 제공하는 VGG16 모델에 대한 상세한 내용은 아래 링크를 참고해 주세요.

텐서플로우 튜토리얼 - VGG16

https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16

In [21]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [22]:
import tensorflow_datasets as tfds

tfds.__version__

'4.9.2'

In [23]:
IMG_SIZE = 160 # 리사이징할 이미지의 크기

In [24]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model VGG16
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                         include_top=False,
                                         weights='imagenet')

간단하죠? 우리는 방금 이미 잘 학습된 모델을 가져왔습니다.
이 모델은 입력된 이미지를 입력받으면 해당 이미지의 특징을 쏙쏙 잘 뽑아낸 특징 벡터를 추출해낼 수 있죠.

실제로 모델에 이미지 데이터가 들어가면 어떻게 되는지 확인해 보겠습니다. 우리가 학습할 때 사용했던 데이터인 image_batch를 넣어보죠.
넣기 전에 image_batch의 원래 사이즈를 다시 확인해봅시다.

In [26]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

image_batch.shape

NameError: name 'train' is not defined

위의 shape는 위에서 많이 확인해봤습니다.
다시 한번 짚어보면, 32는 이미지의 장수, 즉 32장의 이미지가 들어있다는 뜻이고, 각 이미지의 크기는 (160, 160, 3)이었습니다.
160은 각각 height와 width, 그리고 3은 채널 수로 R, G, B 3개였죠.

그렇다면 모델에 이 배치를 넣으면 shape가 어떻게 바뀔까요? 확인해봅시다.

feature_batch = base_model(image_batch)
feature_batch.shape